In [1]:
source("/home/mount/project/config_sing.R")

── Attaching packages ─────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
dir(FD_RES)

[1] "annotation_fragment" "annotation_segment"  "bedgraph"           
 [4] "bootstrap"           "count_fragment"      "count_segment"      
 [7] "coverage"            "data"                "kmer"               
[10] "log"                 "model_linear"        "source"

## Import library size

In [3]:
get_sample = function(idn_sample){
    idn = idn_sample
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

In [4]:
fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

ctypes = c(col_integer(), col_character())
cnames = c("Size", "Fpath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names = cnames)
dat_lib = dat_lib %>% 
    mutate(Sample = tools::file_path_sans_ext(basename(Fpath))) %>%
    mutate(Group = get_sample(Sample))
dat_lib = dat_lib %>% dplyr::select(Size, Sample, Group)
head(dat_lib, 10)

Size,Sample,Group
<dbl>,<chr>,<chr>
18666630,Input1,Input
20167924,Input2,Input
23280988,Input3,Input
19003938,Input4,Input
15325016,Input5,Input
48376253,TFX2_AZD2906,TFX_AZD2906
52542517,TFX2_AZD9567,TFX_AZD9567
43646484,TFX2_CORT108297,TFX_CORT108297
41732268,TFX2_CpdA,TFX_CpdA


## Import data

In [5]:
### import
ctypes = c(col_character(), col_integer(), col_integer(), col_integer(),
           col_character(), col_integer(), col_integer(),
           col_character(), col_double(),  col_integer())
cnames = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
           "Chrom_MTF",  "Start_MTF",  "End_MTF",
           "Motif", "Score", "Overlap")

In [6]:
### set samples
Samples=c(
    paste0("Input", 1:5),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex")
)

-----

In [7]:
fdiry = file.path(FD_RES, "annotation_fragment", "Input1")
fname = "chr21.bed"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, col_types=ctypes, col_names=cnames)
print(head(dat, 3))

# A tibble: 3 × 10
  Chrom_Frag Start_Frag End_Frag Count_Frag Chrom_MTF Start_MTF End_MTF Motif   
  <chr>           <dbl>    <dbl>      <dbl> <chr>         <dbl>   <dbl> <chr>   
1 chr21         5032566  5033537          1 chr21       5032551 5032569 NR/8    
2 chr21         5032566  5033537          1 chr21       5032560 5032569 CREB/AT…
3 chr21         5032566  5033537          1 chr21       5032560 5032578 ZNF140  
# … with 2 more variables: Score <dbl>, Overlap <dbl>


In [10]:
colnames(dat)

[1] "Chrom_Frag" "Start_Frag" "End_Frag"   "Count_Frag" "Chrom_MTF" 
 [6] "Start_MTF"  "End_MTF"    "Motif"      "Score"      "Overlap"

In [9]:
dim(dat)

[1] 42573746       10

In [11]:
dat %>% dplyr::filter(Score >= 8.7) %>% dim

[1] 9294333      10

-----

In [1]:
print(1)

[1] 1


In [7]:
lst_dat = lapply(Samples, function(sam){
    fdiry = file.path(FD_RES, "annotation_fragment", sam)
    fname = "chr21.bed"
    fpath = file.path(fdiry, fname)
    dat = read_tsv(fpath, col_types=ctypes, col_names=cnames)
    dat = dat %>% dplyr::filter(Score >= 8.7)
    dat$Sample = sam
    return(dat)
})

In [8]:
### change motif names
lst_dat = lapply(lst_dat, function(dat){
    dat$Motif = str_replace_all(
        string  = dat$Motif, 
        pattern = "/", 
        replacement = "_")
    return(dat)
})

In [9]:
lapply(lst_dat, dim)

[[1]]
[1] 9294333      11

[[2]]
[1] 10049822       11

[[3]]
[1] 11704794       11

[[4]]
[1] 9479907      11

[[5]]
[1] 7605868      11

[[6]]
[1] 22801075       11

[[7]]
[1] 14670638       11

[[8]]
[1] 17284924       11

[[9]]
[1] 16037677       11

[[10]]
[1] 23965680       11

[[11]]
[1] 15064299       11

[[12]]
[1] 20120112       11

[[13]]
[1] 24417143       11

In [10]:
lapply(lst_dat, head, 2)

Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Chrom_MTF,Start_MTF,End_MTF,Motif,Score,Overlap,Sample
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
chr21,5032566,5033537,1,chr21,5032561,5032570,SREBF1,10.6886,4,Input1
chr21,5032566,5033537,1,chr21,5032581,5032600,ZFN121,19.3355,19,Input1
Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Chrom_MTF,Start_MTF,End_MTF,Motif,Score,Overlap,Sample
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
chr21,5030847,5031732,1,chr21,5030838,5030868,GC-tract,10.11408,21,Input2
chr21,5030847,5031732,1,chr21,5030843,5030852,TFAP2_1,9.15790,5,Input2
Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Chrom_MTF,Start_MTF,End_MTF,Motif,Score,Overlap,Sample
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
chr21,5032523,5033388,1,chr21,5032498,5032553,KLF_SP_2,8.965320,30,Input3


In [11]:
dat_chr21 = bind_rows(lst_dat)

In [12]:
dim(dat_chr21)

[1] 202496272        11

In [17]:
dat_chr21_90 = dat_chr21 %>% dplyr::filter(Score >= 9.84)

In [18]:
dim(dat_chr21_90)

[1] 67216604       11

In [13]:
dat_chr21_95 = dat_chr21 %>% dplyr::filter(Score >= 10.81)

In [14]:
dim(dat_chr21_95)

[1] 26003205       11

In [20]:
fdiry = file.path(FD_RES, "annotation_fragment")
fname = "motif_annotation_chr21_090.bed"
fpath = file.path(fdiry, fname)
write_tsv(dat_chr21_90, fpath)

In [15]:
fdiry = file.path(FD_RES, "annotation_fragment")
fname = "motif_annotation_chr21_095.bed"
fpath = file.path(fdiry, fname)
write_tsv(dat_chr21_95, fpath)